# Crawl dataset with all submissions info
OpenReview Venue Crawling

In [6]:
%load_ext autoreload
%autoreload 2

import time
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm
from tqdm.notebook import tqdm
import requests
import openreview
import json
import numpy as np
import os
# from get_paper_data import get_paper_data_multi
import json
os.environ['ALL_PROXY'] = 'http://127.0.0.1:7890'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Crawl list of all submissions
Here we scrape the _notes_ , (list of all submissions) using OpenReview's API, way faster than Selenium-based scraping.


In [7]:
client = openreview.api.OpenReviewClient(
    baseurl='https://api2.openreview.net',
    username="",
    password=""
)

In [8]:
venue_id = 'NeurIPS.cc/2024/Conference'
venue_group = client.get_group(venue_id)
submission_name = venue_group.content['submission_name']['value']
submissions = client.get_all_notes(invitation=f'{venue_id}/-/{submission_name}', details='directReplies')

Retrying request: GET /groups?id=NeurIPS.cc%2F2024%2FConference, response: no response, error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


Getting V2 Notes: 100%|█████████▉| 4233/4238 [00:10<00:00, 414.35it/s]


In [9]:
def submission2note(submission, idx):
    # review_keys=['summary', 'strengths', 'weaknesses', 'questions']
    # total_replies = submission.details["directReplies"]
    content = submission.content
    # rating_replies = [reply for reply in submission.details["directReplies"] if "rating" in reply["content"]]
    # reply = 'no decision'
    # for rep in submission.details['directReplies']:
    #     if any(invitation.endswith(f'/-/{decision_invitation_name}') for invitation in rep['invitations']):
    #         reply = rep['content']['decision']['value']
    # ratings = [
    #         int(reply["content"]["rating"]["value"][0])
    #         for reply in rating_replies
    # ]
    ratings = [0.0]
    rating = "0.00"
    variance = "0.00"
    authors = content["authors"]["value"] if "authors" in content else []
    decision = content['venue']['value'].split(' ')[-1].lower()
    decision = f"Accept ({decision})"
    note = {
        "id": submission.id,
        "decision": decision,
        "authors": authors,
        "emails": [],
        "rank": idx,
        "title": content["title"]["value"],
        "ratings": ratings,
        "rating": rating,
        "confidences": [0.0],
        "variance": variance,
        "withdraw": 1 if "Withdrawn" in submission.content["venue"]["value"] else 0,
        "abstract": content["abstract"]["value"],
        "url": f"https://openreview.net/forum?id={submission.id}",

    }
    return note

In [10]:
notes = [submission2note(submission, idx) for idx, submission in enumerate(submissions)]
with open('data/neurips2024.json', 'w') as f:
    json.dump(notes, f)